### Dashboard interativo com KPIs, gráficos e insights do negócio

In [0]:
from pyspark.sql.functions import *
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# CONFIGURAÇÃO

In [0]:
spark.sql("USE CATALOG hive_metastore")
spark.sql("USE healthcare_gold")

print("="*80)
print("HEALTHCARE ANALYTICS DASHBOARD")
print("="*80)

## FUNÇÃO AUXILIAR PARA CONVERTER SPARK DF PARA PANDAS

In [0]:
def spark_to_pandas(table_name):
    """Converte tabela Spark para Pandas para visualização"""
    return spark.table(table_name).toPandas()
    
displayHTML("""
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
            padding: 40px; 
            border-radius: 15px; 
            margin-bottom: 30px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.3);">
    <h1 style="color: white; font-size: 48px; margin: 0; text-align: center; font-weight: 700;">
        🏥 Healthcare Analytics
    </h1>
    <p style="color: rgba(255,255,255,0.9); font-size: 20px; text-align: center; margin-top: 10px;">
        Dashboard Executivo - Análise de Consultas Médicas
    </p>
</div>
""")

In [0]:
# ============================================================================
# SEÇÃO 1: KPIs PRINCIPAIS
# ============================================================================

In [0]:
displayHTML("<h2 style='color: #667eea; font-size: 32px; margin-top: 40px;'>📊 KPIs Principais</h2>")

df_periodo = spark_to_pandas("agg_consultas_por_periodo")

total_consultas = df_periodo['total_consultas'].sum()
total_receita = df_periodo['receita_total'].sum()
ticket_medio = df_periodo['ticket_medio'].mean()
total_pacientes = df_periodo['pacientes_unicos'].sum()
total_medicos = df_periodo['medicos_ativos'].sum()

displayHTML(f"""
<div style="display: grid; grid-template-columns: repeat(5, 1fr); gap: 20px; margin: 30px 0;">
    
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                padding: 25px; 
                border-radius: 12px; 
                box-shadow: 0 5px 15px rgba(0,0,0,0.2);
                text-align: center;">
        <div style="color: rgba(255,255,255,0.8); font-size: 14px; font-weight: 600;">CONSULTAS</div>
        <div style="color: white; font-size: 36px; font-weight: 700; margin: 10px 0;">{total_consultas:,}</div>
        <div style="color: rgba(255,255,255,0.7); font-size: 12px;">Total realizadas</div>
    </div>
    
    <div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); 
                padding: 25px; 
                border-radius: 12px; 
                box-shadow: 0 5px 15px rgba(0,0,0,0.2);
                text-align: center;">
        <div style="color: rgba(255,255,255,0.8); font-size: 14px; font-weight: 600;">RECEITA</div>
        <div style="color: white; font-size: 36px; font-weight: 700; margin: 10px 0;">R$ {total_receita:,.0f}</div>
        <div style="color: rgba(255,255,255,0.7); font-size: 12px;">Receita total</div>
    </div>
    
    <div style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); 
                padding: 25px; 
                border-radius: 12px; 
                box-shadow: 0 5px 15px rgba(0,0,0,0.2);
                text-align: center;">
        <div style="color: rgba(255,255,255,0.8); font-size: 14px; font-weight: 600;">TICKET MÉDIO</div>
        <div style="color: white; font-size: 36px; font-weight: 700; margin: 10px 0;">R$ {ticket_medio:.0f}</div>
        <div style="color: rgba(255,255,255,0.7); font-size: 12px;">Por consulta</div>
    </div>
    
    <div style="background: linear-gradient(135deg, #43e97b 0%, #38f9d7 100%); 
                padding: 25px; 
                border-radius: 12px; 
                box-shadow: 0 5px 15px rgba(0,0,0,0.2);
                text-align: center;">
        <div style="color: rgba(255,255,255,0.8); font-size: 14px; font-weight: 600;">PACIENTES</div>
        <div style="color: white; font-size: 36px; font-weight: 700; margin: 10px 0;">{total_pacientes:,}</div>
        <div style="color: rgba(255,255,255,0.7); font-size: 12px;">Únicos atendidos</div>
    </div>
    
    <div style="background: linear-gradient(135deg, #fa709a 0%, #fee140 100%); 
                padding: 25px; 
                border-radius: 12px; 
                box-shadow: 0 5px 15px rgba(0,0,0,0.2);
                text-align: center;">
        <div style="color: rgba(255,255,255,0.8); font-size: 14px; font-weight: 600;">MÉDICOS</div>
        <div style="color: white; font-size: 36px; font-weight: 700; margin: 10px 0;">{total_medicos:,}</div>
        <div style="color: rgba(255,255,255,0.7); font-size: 12px;">Ativos no período</div>
    </div>
    
</div>
""")


In [0]:
# ============================================================================
# SEÇÃO 2: EVOLUÇÃO TEMPORAL
# ============================================================================

In [0]:
displayHTML("<h2 style='color: #667eea; font-size: 32px; margin-top: 50px;'>📈 Evolução Temporal</h2>")

df_periodo_sorted = df_periodo.sort_values(['ano', 'mes'])
df_periodo_sorted['periodo'] = df_periodo_sorted['ano'].astype(str) + '-' + df_periodo_sorted['mes'].astype(str).str.zfill(2)

fig1 = make_subplots(specs=[[{"secondary_y": True}]])

fig1.add_trace(
    go.Scatter(
        x=df_periodo_sorted['periodo'],
        y=df_periodo_sorted['total_consultas'],
        name="Consultas",
        mode='lines+markers',
        line=dict(color='#667eea', width=3),
        marker=dict(size=8)
    ),
    secondary_y=False
)

fig1.add_trace(
    go.Scatter(
        x=df_periodo_sorted['periodo'],
        y=df_periodo_sorted['receita_total'],
        name="Receita (R$)",
        mode='lines+markers',
        line=dict(color='#f5576c', width=3),
        marker=dict(size=8)
    ),
    secondary_y=True
)

fig1.update_layout(
    title="Evolução de Consultas e Receita por Mês",
    hovermode='x unified',
    height=400,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)'
)

fig1.update_xaxes(title_text="Período")
fig1.update_yaxes(title_text="Nº de Consultas", secondary_y=False)
fig1.update_yaxes(title_text="Receita (R$)", secondary_y=True)

fig1.show()


In [0]:
df_periodo_sorted['perc_plano'] = (df_periodo_sorted['consultas_plano'] / df_periodo_sorted['total_consultas'] * 100)
df_periodo_sorted['perc_particular'] = 100 - df_periodo_sorted['perc_plano']

fig2 = go.Figure()

fig2.add_trace(go.Bar(
    x=df_periodo_sorted['periodo'],
    y=df_periodo_sorted['perc_plano'],
    name='Plano de Saúde',
    marker_color='#43e97b'
))

fig2.add_trace(go.Bar(
    x=df_periodo_sorted['periodo'],
    y=df_periodo_sorted['perc_particular'],
    name='Particular',
    marker_color='#f093fb'
))

fig2.update_layout(
    title="Distribuição: Plano de Saúde vs Particular (%)",
    barmode='stack',
    height=400,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    yaxis_title="Percentual (%)"
)

fig2.show()

In [0]:
# ============================================================================
# SEÇÃO 3: ANÁLISE POR ESPECIALIDADE
# ============================================================================

In [0]:
displayHTML("<h2 style='color: #667eea; font-size: 32px; margin-top: 50px;'>👨‍⚕️ Análise por Especialidade</h2>")

df_especialidade = spark_to_pandas("agg_resumo_especialidade")
df_especialidade_top = df_especialidade.nlargest(10, 'receita_total')

fig3 = go.Figure(data=[
    go.Bar(
        y=df_especialidade_top['especialidade'],
        x=df_especialidade_top['receita_total'],
        orientation='h',
        marker=dict(
            color=df_especialidade_top['receita_total'],
            colorscale='Viridis',
            showscale=True
        ),
        text=df_especialidade_top['receita_total'].apply(lambda x: f'R$ {x:,.0f}'),
        textposition='auto'
    )
])

fig3.update_layout(
    title="Top 10 Especialidades por Receita",
    xaxis_title="Receita (R$)",
    yaxis_title="Especialidade",
    height=500,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)'
)

fig3.show()

In [0]:
fig4 = px.pie(
    df_especialidade_top,
    values='total_consultas',
    names='especialidade',
    title='Distribuição de Consultas por Especialidade',
    color_discrete_sequence=px.colors.sequential.RdBu
)

fig4.update_traces(textposition='inside', textinfo='percent+label')
fig4.update_layout(height=500)

fig4.show()

In [0]:
# ============================================================================
# SEÇÃO 4: ANÁLISE DE MÉDICOS
# ============================================================================

In [0]:
displayHTML("<h2 style='color: #667eea; font-size: 32px; margin-top: 50px;'>🏆 Top Médicos</h2>")

df_medicos = spark_to_pandas("agg_consultas_por_medico")
df_top_medicos = df_medicos.nlargest(15, 'receita_gerada')

fig5 = go.Figure()

fig5.add_trace(go.Bar(
    y=df_top_medicos['especialidade'],
    x=df_top_medicos['receita_gerada'],
    orientation='h',
    marker=dict(
        color=df_top_medicos['total_consultas'],
        colorscale='Blues',
        showscale=True,
        colorbar=dict(title="Consultas")
    ),
    text=df_top_medicos['receita_gerada'].apply(lambda x: f'R$ {x:,.0f}'),
    textposition='auto',
    hovertemplate='<b>%{y}</b><br>Receita: R$ %{x:,.0f}<br>Consultas: %{marker.color}<extra></extra>'
))

fig5.update_layout(
    title="Top 15 Médicos por Receita Gerada",
    xaxis_title="Receita (R$)",
    yaxis_title="Especialidade",
    height=600,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)'
)

fig5.show()


In [0]:
# ============================================================================
# SEÇÃO 5: ANÁLISE DE CLÍNICAS
# ============================================================================

In [0]:
displayHTML("<h2 style='color: #667eea; font-size: 32px; margin-top: 50px;'>🏥 Análise de Clínicas</h2>")

df_clinicas = spark_to_pandas("agg_consultas_por_clinica")

fig6 = px.bar(
    df_clinicas,
    x='tipo_clinica',
    y='receita_total',
    color='estado',
    title='Receita por Tipo de Clínica e Estado',
    labels={'receita_total': 'Receita (R$)', 'tipo_clinica': 'Tipo de Clínica'},
    height=500,
    color_discrete_sequence=px.colors.qualitative.Set3
)

fig6.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)'
)

fig6.show()



In [0]:
fig7 = px.scatter(
    df_clinicas,
    x='total_consultas',
    y='receita_total',
    size='medicos_atuantes',
    color='tipo_clinica',
    hover_data=['estado', 'consultas_por_medico'],
    title='Eficiência das Clínicas: Consultas vs Receita',
    labels={
        'total_consultas': 'Total de Consultas',
        'receita_total': 'Receita Total (R$)',
        'medicos_atuantes': 'Nº Médicos'
    },
    height=500
)

fig7.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)'
)

fig7.show()

In [0]:
# ============================================================================
# SEÇÃO 6: ANÁLISE DE DIAGNÓSTICOS
# ============================================================================

In [0]:
displayHTML("<h2 style='color: #667eea; font-size: 32px; margin-top: 50px;'>🩺 Análise de Diagnósticos (CID)</h2>")

df_diagnostico = spark_to_pandas("agg_consultas_por_diagnostico")
df_top_diagnosticos = df_diagnostico.nlargest(15, 'total_casos')

fig8 = go.Figure()

fig8.add_trace(go.Bar(
    x=df_top_diagnosticos['total_casos'],
    y=df_top_diagnosticos['descricao_cid'],
    orientation='h',
    marker=dict(
        color=df_top_diagnosticos['custo_total'],
        colorscale='Reds',
        showscale=True,
        colorbar=dict(title="Custo Total")
    ),
    text=df_top_diagnosticos['codigo_cid'],
    textposition='inside',
    hovertemplate='<b>%{y}</b><br>Casos: %{x}<br>CID: %{text}<br>Custo: R$ %{marker.color:,.0f}<extra></extra>'
))

fig8.update_layout(
    title="Top 15 Diagnósticos por Número de Casos",
    xaxis_title="Total de Casos",
    yaxis_title="Diagnóstico (CID)",
    height=600,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)'
)

fig8.show()

In [0]:
# ============================================================================
# SEÇÃO 7: PERFIL DE PACIENTES
# ============================================================================

In [0]:
displayHTML("<h2 style='color: #667eea; font-size: 32px; margin-top: 50px;'>👥 Perfil de Pacientes</h2>")

df_perfil = spark_to_pandas("agg_perfil_pacientes")

df_faixa = df_perfil.groupby('faixa_etaria').agg({
    'total_pacientes': 'sum',
    'total_consultas': 'sum'
}).reset_index()

fig9 = make_subplots(specs=[[{"secondary_y": True}]])

fig9.add_trace(
    go.Bar(
        x=df_faixa['faixa_etaria'],
        y=df_faixa['total_pacientes'],
        name="Pacientes",
        marker_color='#4facfe'
    ),
    secondary_y=False
)

fig9.add_trace(
    go.Scatter(
        x=df_faixa['faixa_etaria'],
        y=df_faixa['total_consultas'],
        name="Consultas",
        mode='lines+markers',
        line=dict(color='#f5576c', width=3),
        marker=dict(size=10)
    ),
    secondary_y=True
)

fig9.update_layout(
    title="Distribuição de Pacientes e Consultas por Faixa Etária",
    xaxis_title="Faixa Etária",
    height=500,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)'
)

fig9.update_yaxes(title_text="Nº de Pacientes", secondary_y=False)
fig9.update_yaxes(title_text="Nº de Consultas", secondary_y=True)

fig9.show()

In [0]:
df_sexo = df_perfil.groupby('sexo').agg({
    'total_pacientes': 'sum',
    'gasto_total': 'sum'
}).reset_index()

fig10 = px.pie(
    df_sexo,
    values='total_pacientes',
    names='sexo',
    title='Distribuição de Pacientes por Sexo',
    color_discrete_map={'M': '#4facfe', 'F': '#f093fb', 'O': '#43e97b'}
)

fig10.update_traces(textposition='inside', textinfo='percent+label+value')
fig10.update_layout(height=400)

fig10.show()

In [0]:
# ============================================================================
# SEÇÃO 8: INSIGHTS E RECOMENDAÇÕES
# ============================================================================

In [0]:
displayHTML("<h2 style='color: #667eea; font-size: 32px; margin-top: 50px;'>💡 Insights e Recomendações</h2>")

especialidade_top = df_especialidade.nlargest(1, 'receita_total').iloc[0]
medico_top = df_medicos.nlargest(1, 'receita_gerada').iloc[0]
diagnostico_top = df_diagnostico.nlargest(1, 'total_casos').iloc[0]
taxa_plano = (df_periodo['consultas_plano'].sum() / df_periodo['total_consultas'].sum() * 100)

displayHTML(f"""
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
            padding: 30px; 
            border-radius: 15px; 
            color: white;
            margin: 20px 0;">
    
    <h3 style="margin-top: 0; font-size: 24px;">📊 Principais Insights:</h3>
    
    <div style="margin: 20px 0; padding: 15px; background: rgba(255,255,255,0.1); border-radius: 8px;">
        <strong>🏆 Especialidade Líder:</strong> {especialidade_top['especialidade']}<br>
        Receita: R$ {especialidade_top['receita_total']:,.2f} | 
        Consultas: {especialidade_top['total_consultas']:,} | 
        Médicos: {especialidade_top['medicos_ativos']}
    </div>
    
    <div style="margin: 20px 0; padding: 15px; background: rgba(255,255,255,0.1); border-radius: 8px;">
        <strong>👨‍⚕️ Médico com Maior Receita:</strong> {medico_top['especialidade']}<br>
        Receita: R$ {medico_top['receita_gerada']:,.2f} | 
        Pacientes: {medico_top['pacientes_atendidos']:,} | 
        Ticket Médio: R$ {medico_top['ticket_medio']:.2f}
    </div>
    
    <div style="margin: 20px 0; padding: 15px; background: rgba(255,255,255,0.1); border-radius: 8px;">
        <strong>🩺 Diagnóstico Mais Comum:</strong> {diagnostico_top['descricao_cid']} ({diagnostico_top['codigo_cid']})<br>
        Casos: {diagnostico_top['total_casos']:,} | 
        Custo Médio: R$ {diagnostico_top['custo_medio_tratamento']:.2f}
    </div>
    
    <div style="margin: 20px 0; padding: 15px; background: rgba(255,255,255,0.1); border-radius: 8px;">
        <strong>💳 Taxa de Cobertura por Plano:</strong> {taxa_plano:.1f}%<br>
        {df_periodo['consultas_plano'].sum():,} consultas cobertas por plano de saúde
    </div>
    
    <h3 style="margin-top: 30px; font-size: 24px;">🎯 Recomendações Estratégicas:</h3>
    
    <ul style="line-height: 2; font-size: 16px;">
        <li>🔹 Expandir corpo clínico nas especialidades de maior demanda</li>
        <li>🔹 Implementar programa de fidelização para pacientes de alto valor</li>
        <li>🔹 Otimizar agenda dos médicos de maior performance</li>
        <li>🔹 Desenvolver campanhas preventivas para diagnósticos mais frequentes</li>
        <li>🔹 Negociar convênios para aumentar taxa de cobertura por planos</li>
    </ul>
    
</div>
""")